In [ ]:
import numpy as np
import math
from random import *

# PART I

## Decision Tree w/ CART algorithm

In [2]:
# basic ds = tree structure node...

class NODE:
    def __init__(self, val, idx):
        self.val = val
        self.idx = idx
        self.sign = 0
        self.left = None
        self.right = None

In [27]:
# CART algorithm

class CART:
    def __init__(self, train_data_path, test_data_path, show = 0):
        # data loading
        dta = np.loadtxt(train_data_path, dtype=np.float, delimiter=" ")
        numrow, numcol = np.shape(dta)

        feature = dta[:,0:numcol-1]
        label = dta[:,numcol-1]
        self.model = self.DoingCART(feature,label)
        if show == 1:
            print("Ein =",self.predCART(self.model, feature, label))
            tdta = np.loadtxt(test_data_path, dtype=np.float, delimiter=" ")
            tnumrow, tnumcol = np.shape(tdta)
            tfeature = tdta[:,0:tnumcol-1]
            tlabel = tdta[:,tnumcol-1]
            print("Eout =",self.predCART(self.model, tfeature, tlabel))
    
    def ginical(self, label):
        labelsize = np.shape(label)[0]
        if labelsize == 0:
            return 0
        else:
            return 1-(sum(label==1)/float(labelsize))**2-(sum(label==-1)/float(labelsize))**2
    
    def decisionStump(self, targetfeature, targetlabel):
        '''
        calculation of thetas, only n-1 thetas (not n+1)
        (1) one of theta is inherent in cart_function (base case)
        (2) the other theta is equivilent to -(1): theta x,x,x,x,x == x,x,x,x,x theta
        '''
        thetaArray = np.array([(targetfeature[i] + targetfeature[i + 1]) / 2 for i in range(0, targetfeature.shape[0] - 1)])
        
        currentBranchCriteria = float("inf")
        targetTheta = 0.0
        for theta in thetaArray:
            #np.where retures idx
            LHS = targetlabel[np.where(targetfeature < theta)]
            RHS = targetlabel[np.where(targetfeature >= theta)]
            b = LHS.shape[0] * self.ginical(LHS) + RHS.shape[0] * self.ginical(RHS)
            if currentBranchCriteria > b:
                currentBranchCriteria = b
                targetTheta = theta
        return currentBranchCriteria, targetTheta
    
    def branch(self, feature, label):
        sort_of_each_feature_idx = []
        for i in range(feature.shape[1]):
            sort_of_each_feature_idx.append(np.argsort(feature[:,i]))
        
        bestBranch = float("inf")
        bestIndex = -1
        bestBranchVal = 0
        for i in range(feature.shape[1]): # for each sorted feature
            targetfeature = feature[sort_of_each_feature_idx[i],i]
            targetlabel = label[sort_of_each_feature_idx[i]]
            tempb,tempval = self.decisionStump(targetfeature,targetlabel)
            if bestBranch > tempb:
                bestBranch = tempb
                bestIndex = i
                bestBranchVal = tempval
        '''
        let's split
        '''
        LX=feature[np.where(feature[:,bestIndex]<bestBranchVal)]
        LY=label[np.where(feature[:,bestIndex]<bestBranchVal)]
        RX=feature[np.where(feature[:,bestIndex]>=bestBranchVal)]
        RY=label[np.where(feature[:,bestIndex]>=bestBranchVal)]
        
        return LX,LY,RX,RY,bestIndex,bestBranchVal
    
    def DoingCART(self, feature, label):
        if self.ginical(label) == 0:
            leaf = NODE(-1,-1)
            leaf.sign = label[0] #sign
            #print("leaf")
            return leaf
        LX,LY,RX,RY,bestIndex,bestBranchVal = self.branch(feature,label)
        node = NODE(bestBranchVal,bestIndex) #val, idx
        node.left = self.DoingCART(LX,LY) #l
        node.right = self.DoingCART(RX,RY) #r
        return node
    
    def predCART_onesample(self, root, onex):
        if root.idx == -1:
            return root.sign
        if onex[root.idx] < root.val:
            return self.predCART_onesample(root.left, onex)
        else:
            return self.predCART_onesample(root.right, onex)
    
    def predCART(self, root, feature, label):
        count = 0
        for i in range(np.shape(feature)[0]):
            count += 1 if self.predCART_onesample(root, feature[i]) != label[i] else 0
        return float(count)/np.shape(feature)[0]

## Problem 14

my answer = [c]

In [30]:
DT = CART("hw6_train.txt","hw6_test.txt",1)

Ein = 0.0
Eout = 0.166


### Validate *My Answer* with package

In [61]:
from sklearn.tree import DecisionTreeClassifier

# training data
dta = np.loadtxt("hw6_train.txt", dtype=np.float, delimiter=" ")
numrow, numcol = np.shape(dta)
feature = dta[:,0:numcol-1]
label = dta[:,numcol-1]

#testing data
tdta = np.loadtxt("hw6_test.txt", dtype=np.float, delimiter=" ")
tnumrow, tnumcol = np.shape(tdta)
tfeature = tdta[:,0:tnumcol-1]
tlabel = tdta[:,tnumcol-1]

# Fit regression model
regr_1 = DecisionTreeClassifier(max_depth=2000)
regr_1.fit(feature, label)

# Predict
y_1 = regr_1.predict(tfeature)
print("Eout =", sum(y_1!=tlabel)/1000)

Eout = 0.176


# PART II

## Random Forest

In [48]:
# revised CART for random forest
def ginical(label):
    labelsize = np.shape(label)[0]
    if labelsize == 0:
        return 0
    else:
        return 1-(sum(label==1)/float(labelsize))**2-(sum(label==-1)/float(labelsize))**2

def decisionStump(targetfeature, targetlabel):
    '''
    calculation of thetas, only n-1 thetas (not n+1)
    (1) one of theta is inherent in cart_function (base case)
    (2) the other theta is equivilent to -(1): theta x,x,x,x,x == x,x,x,x,x theta
    '''
    thetaArray = np.array([(targetfeature[i] + targetfeature[i + 1]) / 2 for i in range(0, targetfeature.shape[0] - 1)])

    currentBranchCriteria = float("inf")
    targetTheta = 0.0
    for theta in thetaArray:
        #np.where retures idx
        LHS = targetlabel[np.where(targetfeature < theta)]
        RHS = targetlabel[np.where(targetfeature >= theta)]
        b = LHS.shape[0] * ginical(LHS) + RHS.shape[0] * ginical(RHS)
        if currentBranchCriteria > b:
            currentBranchCriteria = b
            targetTheta = theta
    return currentBranchCriteria, targetTheta

def branch(feature, label):
    sort_of_each_feature_idx = []
    for i in range(feature.shape[1]):
        sort_of_each_feature_idx.append(np.argsort(feature[:,i]))

    bestBranch = float("inf")
    bestIndex = -1
    bestBranchVal = 0
    for i in range(feature.shape[1]): # for each sorted feature
        targetfeature = feature[sort_of_each_feature_idx[i],i]
        targetlabel = label[sort_of_each_feature_idx[i]]
        tempb,tempval = decisionStump(targetfeature,targetlabel)
        if bestBranch > tempb:
            bestBranch = tempb
            bestIndex = i
            bestBranchVal = tempval
    '''
    let's split
    '''
    LX=feature[np.where(feature[:,bestIndex]<bestBranchVal)]
    LY=label[np.where(feature[:,bestIndex]<bestBranchVal)]
    RX=feature[np.where(feature[:,bestIndex]>=bestBranchVal)]
    RY=label[np.where(feature[:,bestIndex]>=bestBranchVal)]

    return LX,LY,RX,RY,bestIndex,bestBranchVal

def DoingCART(feature, label):
    if ginical(label) == 0:
        leaf = NODE(-1,-1)
        leaf.sign = label[0] #sign
        #print("leaf")
        return leaf
    LX,LY,RX,RY,bestIndex,bestBranchVal = branch(feature,label)
    node = NODE(bestBranchVal,bestIndex) #val, idx
    node.left = DoingCART(LX,LY) #l
    node.right = DoingCART(RX,RY) #r
    return node

def predCART_onesample(root, onex):
    if root.idx == -1:
        return root.sign
    if onex[root.idx] < root.val:
        return predCART_onesample(root.left, onex)
    else:
        return predCART_onesample(root.right, onex)

def predCART(root, feature, label):
    count = 0
    for i in range(np.shape(feature)[0]):
        count += 1 if predCART_onesample(root, feature[i]) != label[i] else 0
    return float(count)/np.shape(feature)[0]

In [49]:
# bootstrapmatrix row:T col:idx

bootstrapMatrix = []
for i in range(2000):
    bootstrapMatrix.append(choices(range(0,1000),k=500))

In [67]:
from operator import itemgetter
# training data
dta = np.loadtxt("hw6_train.txt", dtype=np.float, delimiter=" ")
numrow, numcol = np.shape(dta)
trainfeature = dta[:,0:numcol-1]
trainlabel = dta[:,numcol-1]

#testing data
tdta = np.loadtxt("hw6_test.txt", dtype=np.float, delimiter=" ")
tnumrow, tnumcol = np.shape(tdta)
testfeature = tdta[:,0:tnumcol-1]
testlabel = tdta[:,tnumcol-1]

## Problem 15

my answer = [d]

In [69]:
def RF(T):
    error = 0
    forest = []
    for i in range(T):
        x = np.array(list(itemgetter(*bootstrapMatrix[i])(trainfeature)))
        y = np.array(list(itemgetter(*bootstrapMatrix[i])(trainlabel)))
        model = DoingCART(x,y)
        error += predCART(model,testfeature,testlabel)
        forest.append(model)
    return error/float(T), forest

In [70]:
error, forest = RF(2000)
print(error)

0.2341425000000003


In [ ]:
'''def RandomForest(T):
    error = 0
    forest = []
    for i in range(T):
        x = np.array(list(itemgetter(*bootstrapMatrix[i])(trainfeature)))
        y = np.array(list(itemgetter(*bootstrapMatrix[i])(trainlabel)))
        model = DecisionTreeClassifier(max_depth=2000)
        model.fit(x, y)
        
        # Predict
        y_1 = model.predict(testfeature)
        error += sum(y_1!=testlabel)/1000
        forest.append(model)
    return error/float(T), forest

err, forest = RandomForest(2000)'''

## Problem 16

my answer = [a]

In [77]:
def RF_predict(Forest, feature):
    pos=0
    neg=0
    for tree in Forest:
        predict_label = tree.predict([feature])
        if predict_label[0] == 1:
            pos += 1
        else:
            neg += 1
    return (1 if pos > neg else -1)

def cal_RF_error(Forest, feature, label):
    m = np.shape(feature)[0]
    error = 0
    for i in range(m):
        predict_label = RF_predict(Forest, feature[i])
        error += (1 if predict_label!=label[i] else 0)
    return float(error)/m

print(cal_RF_error(forest, trainfeature, trainlabel))

0.016


## Problem 17

my answer = [d]

In [78]:
print(cal_RF_error(forest, testfeature, testlabel))

0.16


## Problem 18

my answer = [b]

In [81]:
def RF_predict_OOB(Forest, feature, idx):
    pos=0
    neg=0
    for key, tree in enumerate(Forest):
        if idx not in bootstrapMatrix[key]:
            predict_label = tree.predict([feature])
            if predict_label[0] == 1:
                pos += 1
            else:
                neg += 1
    return (1 if pos > neg else -1)

def cal_RF_error_OOB(Forest, feature, label):
    m = np.shape(feature)[0]
    error = 0
    for i in range(m):
        predict_label = RF_predict_OOB(Forest, feature[i], i)
        error += (1 if predict_label!=label[i] else 0)
    return float(error)/m

print(cal_RF_error_OOB(forest, trainfeature, trainlabel))

0.075
